In [ ]:
# Importing necessary modules and classes for the main script

from decoder import Decoder  # Importing the Decoder module
from decoderattn import Attention_Decoder  # Importing the Attention_Decoder module
from encoder import Encoder  # Importing the Encoder module
from evaluatemodel import MySeq2SeqModelEvaluation  # Importing the MySeq2SeqModelEvaluation class
from train_with_batch import Seq2SeqTrainer  # Importing the Seq2SeqTrainer module with batch training
from trainmodel import Seq2SeqTrainer  # Importing the Seq2SeqTrainer module for training
from data_proc import *  # Importing necessary functions for data processing


In [15]:
# Importing necessary modules for working with PyTorch and checking GPU availability

import torch  # Importing PyTorch module
from io import open  # Importing the 'open' function from the 'io' module

use_cuda = torch.cuda.is_available()  # Checking if CUDA (GPU) is available


In [16]:
# Setting constants for sequence processing and training

start_token = 0  # Token representing the start of a sequence
end_token = 1  # Token representing the end of a sequence
max_length = 50  # Maximum length of a sequence
num_epochs = 75000  # Number of training epochs


In [30]:
# Initializing data preparation and preparing language pairs

data = DataPreparation('fra', 'eng', True)  # Creating a DataPreparation instance for French to English translation
input_lang, output_lang, pairs = data.prepare_data(reverse=True)  # Preparing language pairs with reverse order



Reading lines...
Form:  ['l or a . dollars l once ?', ' gold ?']
Read 2000 sentence pairs
Trimmed to 1837 sentence pairs
Counting words...
Counted words:
eng 6924
fra 6019


In [18]:
# Setting hidden size and device, and initializing attention decoder and encoder

hidden_size = 256  # Size of the hidden layer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Checking and assigning the device (GPU or CPU)

# Initializing Attention Decoder with specified parameters
attn_decoder = Attention_Decoder(hidden_size, output_lang.n_words, max_length, dropoutP=0.1).to(device)

# Initializing Encoder with specified parameters
encoder = Encoder(input_size=input_lang.n_words, hidden_size=hidden_size, device=device).to(device)

In [ ]:
# Initializing Seq2SeqTrainer and performing custom training

train = Seq2SeqTrainer(encoder, attn_decoder, max_length, start_token, end_token, input_lang, output_lang, device)

# Performing custom training with specified parameters
losses = train.custom_training_function(num_epochs, pairs, print_interval=5000, plot_interval=50, learning_rate=0.01)


: 

In [ ]:
# sauvegarde du modèle
torch.save(encoder,"encoder_bi.pt")
torch.save(attn_decoder,"attn_decoder_bi.pt")

In [ ]:
# chargement du modèle
encoder_loaded = torch.load("encoder_bi.pt")
attn_decoder_loaded = torch.load("attn_decoder_bi.pt")

In [ ]:
# Initializing MySeq2SeqModelEvaluation and evaluating randomly

modele_evaluation = MySeq2SeqModelEvaluation(encoder_loaded, attn_decoder_loaded, max_length, input_lang, output_lang, start_token, end_token, device)

# Evaluating the model randomly on the provided pairs
modele_evaluation.evaluateRandomly(pairs)


In [ ]:
# Generating output and attentions for a given input sentence
output_words, attentions = modele_evaluation.forward("j'ai trop froid .")
print(output_words)